<a href="https://colab.research.google.com/github/stephanedenis/PaniniFS-Research/blob/main/notebooks/colab_dhatu_robust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Dhātu Analysis - Robust Colab Version

Version robuste qui gère les credentials GitHub proprement et évite les erreurs fatales.

In [ ]:
# 📦 Setup robuste avec gestion d'erreurs
import os, json, requests, time, subprocess
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

def setup_git_safely():
    """Configuration Git sécurisée"""
    try:
        # Vérifier si déjà configuré
        result = subprocess.run(['git', 'config', 'user.email'], capture_output=True, text=True)
        if result.returncode != 0 or not result.stdout.strip():
            # Configuration par défaut pour Colab
            subprocess.run(['git', 'config', 'user.email', 'colab@research.panini'], check=True)
            subprocess.run(['git', 'config', 'user.name', 'Colab Research'], check=True)
            print("✅ Git configuré pour Colab")
        else:
            print(f"✅ Git déjà configuré: {result.stdout.strip()}")
            
        return True
    except Exception as e:
        print(f"⚠️ Problème configuration Git: {e}")
        return False

def clone_or_pull_repo():
    """Clone ou pull du repository"""
    REPO_URL = "https://github.com/stephanedenis/PaniniFS-Research"
    
    if not os.path.exists('PaniniFS-Research'):
        print("📥 Clonage du repository...")
        !git clone $REPO_URL
        os.chdir('PaniniFS-Research')
    else:
        print("🔄 Mise à jour du repository...")
        os.chdir('PaniniFS-Research')
        try:
            !git pull origin main
            print("✅ Repository mis à jour")
        except:
            print("⚠️ Mise à jour échouée, continuons avec la version locale")

# Configuration et clonage
git_configured = setup_git_safely()
clone_or_pull_repo()
print("✅ Setup terminé!")

In [ ]:
# 🔍 Analyseur Dhātu Robuste
class RobustDhatuAnalyzer:
    def __init__(self):
        self.dhatu_patterns = {
            'भू': r'भू|bhū|bhuu|être|being|exist',
            'कृ': r'कृ|kṛ|kri|faire|doing|make|create',
            'गम्': r'गम्|gam|aller|going|move|motion',
            'दा': r'दा|dā|daa|donner|giving|give|grant',
            'स्था': r'स्था|sthā|sthaa|être debout|standing|remain',
            'वद्': r'वद्|vad|dire|speak|say|tell',
            'लभ्': r'लभ्|labh|obtenir|obtain|get|receive',
            'पा': r'पा|pā|paa|protéger|protect|guard',
            'हन्': r'हन्|han|tuer|kill|destroy'
        }
        self.results = []
        self.stats = {
            'start_time': datetime.now().isoformat(),
            'total_docs': 0,
            'total_matches': 0
        }
    
    def analyze_text(self, text, source="unknown"):
        """Analyse robuste d'un texte"""
        matches = {}
        try:
            for dhatu, pattern in self.dhatu_patterns.items():
                import re
                count = len(re.findall(pattern, text, re.IGNORECASE))
                if count > 0:
                    matches[dhatu] = count
            
            result = {
                'source': source,
                'text_length': len(text),
                'dhatu_matches': matches,
                'total_matches': sum(matches.values()),
                'timestamp': datetime.now().isoformat(),
                'quality_score': self.calculate_quality(text, matches)
            }
            
            self.results.append(result)
            self.stats['total_docs'] += 1
            self.stats['total_matches'] += result['total_matches']
            
            return result
            
        except Exception as e:
            print(f"⚠️ Erreur analyse: {e}")
            return None
    
    def calculate_quality(self, text, matches):
        """Calcul de qualité du document"""
        score = 0.0
        
        # Bonus longueur
        if 100 <= len(text) <= 2000:
            score += 0.3
        
        # Bonus matches dhātu
        total_matches = sum(matches.values())
        if total_matches > 0:
            score += min(0.4, total_matches * 0.1)
        
        # Bonus diversité dhātu
        if len(matches) > 1:
            score += 0.2
        
        # Bonus mots-clés
        keywords = ['sanskrit', 'grammar', 'verb', 'linguistic']
        for keyword in keywords:
            if keyword.lower() in text.lower():
                score += 0.1
        
        return min(score, 1.0)
    
    def get_summary(self):
        """Résumé complet des analyses"""
        if not self.results:
            return "❌ Aucune analyse effectuée"
        
        avg_quality = sum(r.get('quality_score', 0) for r in self.results) / len(self.results)
        
        return f"""📊 RÉSUMÉ ANALYSE DHĀTU:
├── Documents analysés: {self.stats['total_docs']}
├── Dhātus détectés: {self.stats['total_matches']}
├── Qualité moyenne: {avg_quality:.3f}/1.0
└── Timestamp: {datetime.now().strftime('%H:%M:%S')}"""

# Initialiser l'analyseur robuste
analyzer = RobustDhatuAnalyzer()
print("🔍 Analyseur Dhātu robuste initialisé!")
print(f"🎯 {len(analyzer.dhatu_patterns)} dhātus configurés")

In [ ]:
# 📁 Chargement robuste des données
def load_and_analyze_corpus():
    """Charge et analyse le corpus complet"""
    data_dirs = ['data/incremental_corpus', 'colab_results']
    total_files = 0
    processed_docs = 0
    
    for data_dir in data_dirs:
        if not os.path.exists(data_dir):
            print(f"⚠️ Dossier {data_dir} non trouvé")
            continue
            
        files = [f for f in os.listdir(data_dir) if f.endswith('.json')]
        total_files += len(files)
        
        print(f"📁 {data_dir}: {len(files)} fichiers")
        
        # Traiter les fichiers batch par batch
        for i, filename in enumerate(files[:10]):  # Limite pour éviter surcharge
            try:
                filepath = os.path.join(data_dir, filename)
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                # Analyser selon le format
                documents = []
                if 'documents' in data:
                    documents = data['documents']
                elif isinstance(data, list):
                    documents = data
                elif 'content' in data:
                    documents = [data]
                
                # Analyser chaque document
                for doc in documents[:5]:  # 5 docs par fichier
                    if isinstance(doc, dict):
                        content = doc.get('content', '') + ' ' + doc.get('title', '')
                        source = doc.get('source', filename)
                    else:
                        content = str(doc)
                        source = filename
                    
                    if len(content) > 50:  # Minimum de contenu
                        result = analyzer.analyze_text(content, source)
                        if result and result['total_matches'] > 0:
                            processed_docs += 1
                            title = doc.get('title', 'Sans titre') if isinstance(doc, dict) else 'Document'
                            print(f"✅ {title[:40]}... - {result['total_matches']} dhātus (Q:{result['quality_score']:.2f})")
                            
            except Exception as e:
                print(f"⚠️ Erreur fichier {filename}: {e}")
                continue
    
    print(f"\n📊 BILAN CHARGEMENT:")
    print(f"├── Fichiers trouvés: {total_files}")
    print(f"├── Documents traités: {processed_docs}")
    print(f"└── {analyzer.get_summary()}")
    
    return processed_docs > 0

# Charger et analyser
success = load_and_analyze_corpus()
if not success:
    print("⚠️ Aucune donnée trouvée, analysons du contenu d'exemple...")
    
    # Contenu d'exemple si pas de données
    examples = [
        "Sanskrit grammar involves complex verb conjugations with dhātu roots like भू (to be) and कृ (to do).",
        "Panini's Ashtadhyayi describes how dhātus like गम् (to go) form various tenses and moods.",
        "The verb स्था (to stand) exemplifies the aspectual system in Sanskrit linguistics."
    ]
    
    for i, example in enumerate(examples):
        analyzer.analyze_text(example, f"example_{i+1}")
    
    print(analyzer.get_summary())

In [ ]:
# 📊 Visualisation robuste
def create_dhatu_visualization():
    """Crée des visualisations des résultats"""
    if not analyzer.results:
        print("❌ Pas de données à visualiser")
        return
    
    try:
        # Agrégation des données
        dhatu_counts = {}
        quality_scores = []
        
        for result in analyzer.results:
            for dhatu, count in result['dhatu_matches'].items():
                dhatu_counts[dhatu] = dhatu_counts.get(dhatu, 0) + count
            quality_scores.append(result.get('quality_score', 0))
        
        # Graphique 1: Distribution des dhātus
        if dhatu_counts:
            plt.figure(figsize=(12, 5))
            
            # Subplot 1: Dhātus
            plt.subplot(1, 2, 1)
            dhatus = list(dhatu_counts.keys())
            counts = list(dhatu_counts.values())
            
            bars = plt.bar(dhatus, counts, color='skyblue', alpha=0.7)
            plt.title(f'Distribution des Dhātus ({sum(counts)} total)')
            plt.ylabel('Occurrences')
            plt.xticks(rotation=45)
            
            # Ajouter valeurs sur les barres
            for bar, count in zip(bars, counts):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                        str(count), ha='center', va='bottom')
            
            # Subplot 2: Qualité
            plt.subplot(1, 2, 2)
            plt.hist(quality_scores, bins=10, color='lightgreen', alpha=0.7, edgecolor='black')
            plt.title(f'Distribution Qualité (μ={sum(quality_scores)/len(quality_scores):.2f})')
            plt.xlabel('Score de qualité')
            plt.ylabel('Nombre de documents')
            
            plt.tight_layout()
            plt.show()
            
            # Statistiques détaillées
            print("\n🏆 TOP DHĀTUS:")
            sorted_dhatus = sorted(dhatu_counts.items(), key=lambda x: x[1], reverse=True)
            for dhatu, count in sorted_dhatus[:5]:
                percentage = (count / sum(counts)) * 100
                print(f"  {dhatu}: {count} occurrences ({percentage:.1f}%)")
            
            print(f"\n📈 QUALITÉ MOYENNE: {sum(quality_scores)/len(quality_scores):.3f}/1.0")
            
        else:
            print("⚠️ Aucun dhātu détecté pour la visualisation")
            
    except Exception as e:
        print(f"❌ Erreur visualisation: {e}")

# Créer la visualisation
create_dhatu_visualization()

In [ ]:
# 🤖 Système de feedback robuste
def create_and_save_feedback():
    """Crée et sauvegarde le feedback pour le collecteur"""
    if not analyzer.results:
        print("⚠️ Pas assez de données pour créer un feedback")
        return False
    
    try:
        # Calculer métriques avancées
        total_docs = len(analyzer.results)
        total_matches = sum(r['total_matches'] for r in analyzer.results)
        avg_quality = sum(r.get('quality_score', 0) for r in analyzer.results) / total_docs
        
        # Analyser les sources performantes
        source_performance = {}
        for result in analyzer.results:
            source = result['source']
            if source not in source_performance:
                source_performance[source] = {'docs': 0, 'matches': 0, 'quality': 0}
            
            source_performance[source]['docs'] += 1
            source_performance[source]['matches'] += result['total_matches']
            source_performance[source]['quality'] += result.get('quality_score', 0)
        
        # Moyennes par source
        for source in source_performance:
            perf = source_performance[source]
            perf['avg_matches'] = perf['matches'] / perf['docs']
            perf['avg_quality'] = perf['quality'] / perf['docs']
        
        # Recommandations intelligentes
        recommendations = {
            'increase_batch_size': avg_quality > 0.6,
            'focus_on_sanskrit': total_matches / total_docs > 2,
            'diversify_sources': len(source_performance) < 3,
            'maintain_quality': avg_quality > 0.5
        }
        
        # Sources à privilégier
        best_sources = sorted(
            [(s, p['avg_quality']) for s, p in source_performance.items()],
            key=lambda x: x[1], reverse=True
        )[:3]
        
        feedback = {
            'timestamp': datetime.now().isoformat(),
            'colab_analysis': {
                'documents_processed': total_docs,
                'dhatu_matches_found': total_matches,
                'average_quality': round(avg_quality, 3),
                'processing_rate': f'{total_docs}_docs_analyzed',
                'analysis_depth': 'robust_multi_dhatu'
            },
            'collector_recommendations': {
                'priority_sources': [s[0] for s in best_sources],
                'target_quality_threshold': max(0.5, avg_quality - 0.1),
                'suggested_batch_size': 20 if recommendations['increase_batch_size'] else 15,
                'focus_areas': ['sanskrit_texts', 'linguistic_papers'] if recommendations['focus_on_sanskrit'] else ['general_linguistics']
            },
            'performance_metrics': {
                'colab_processing_speed': 'optimal',
                'data_consumption_rate': 'high',
                'buffer_needs': 'continuous_feeding_required',
                'next_analysis_ready': True
            },
            'source_analysis': source_performance
        }
        
        # Sauvegarder le feedback
        os.makedirs('colab_results', exist_ok=True)
        feedback_file = 'colab_results/colab_feedback.json'
        
        with open(feedback_file, 'w', encoding='utf-8') as f:
            json.dump(feedback, f, ensure_ascii=False, indent=2)
        
        print(f"💾 Feedback sauvé: {feedback_file}")
        
        # Tentative de synchronisation GitHub (robuste)
        return sync_to_github_safely(feedback_file)
        
    except Exception as e:
        print(f"❌ Erreur création feedback: {e}")
        return False

def sync_to_github_safely(feedback_file):
    """Synchronisation GitHub robuste"""
    try:
        # Vérifier si Git est configuré
        result = subprocess.run(['git', 'config', 'user.email'], capture_output=True, text=True)
        
        if result.returncode != 0 or not result.stdout.strip():
            print("ℹ️ Git non configuré - feedback sauvé localement uniquement")
            print("💡 Pour activer GitHub, exécutez:")
            print("   !git config user.email 'votre-email@example.com'")
            print("   !git config user.name 'Votre Nom'")
            return False
        
        # Vérifier si on peut faire des commits
        test_result = subprocess.run(['git', 'status'], capture_output=True, text=True)
        if test_result.returncode != 0:
            print("⚠️ Repository Git non accessible")
            return False
        
        # Tentative de commit
        subprocess.run(['git', 'add', feedback_file], check=True, capture_output=True)
        
        commit_result = subprocess.run(
            ['git', 'commit', '-m', '🔄 Colab feedback: analyse dhātu robuste'],
            capture_output=True, text=True
        )
        
        if commit_result.returncode == 0:
            # Tentative de push (peut échouer selon les credentials)
            push_result = subprocess.run(
                ['git', 'push', 'origin', 'main'],
                capture_output=True, text=True, timeout=30
            )
            
            if push_result.returncode == 0:
                print("🚀 Feedback synchronisé sur GitHub!")
                return True
            else:
                print("💾 Feedback commité localement (push échoué)")
                print("ℹ️ Synchronisation manuelle requise")
                return False
        else:
            print("💾 Feedback sauvé (rien de nouveau à commiter)")
            return True
            
    except subprocess.TimeoutExpired:
        print("⏰ Timeout synchronisation GitHub")
        return False
    except Exception as e:
        print(f"⚠️ Erreur synchronisation: {e}")
        print("💾 Feedback reste sauvé localement")
        return False

# Créer et sauvegarder le feedback
feedback_success = create_and_save_feedback()

if feedback_success:
    print("\n✅ FEEDBACK ENVOYÉ AU COLLECTEUR")
    print("🤖 Le collecteur va adapter sa stratégie")
else:
    print("\n💾 Feedback sauvé localement")
    print("🔄 Synchronisation manuelle possible plus tard")

In [ ]:
# 🔄 Mode continu robuste (optionnel)
def run_continuous_analysis_robust(duration_minutes=10, check_interval=30):
    """Mode d'analyse continu avec gestion d'erreurs"""
    print(f"🔄 Mode continu démarré: {duration_minutes}min (vérification toutes les {check_interval}s)")
    
    start_time = time.time()
    cycles = 0
    last_file_count = 0
    
    try:
        while (time.time() - start_time) < (duration_minutes * 60):
            cycles += 1
            print(f"\n🔍 Cycle {cycles} - {datetime.now().strftime('%H:%M:%S')}")
            
            # Vérifier nouvelles données
            try:
                current_file_count = len([f for f in os.listdir('data/incremental_corpus') 
                                        if f.endswith('.json')])
                
                if current_file_count > last_file_count:
                    new_files = current_file_count - last_file_count
                    print(f"📁 {new_files} nouveaux fichiers détectés")
                    
                    # Analyser les nouvelles données
                    success = load_and_analyze_corpus()
                    
                    if success:
                        # Créer nouveau feedback
                        create_and_save_feedback()
                        print(f"✅ Feedback mis à jour (cycle {cycles})")
                    
                    last_file_count = current_file_count
                    
                else:
                    print(f"📊 Pas de nouvelles données ({current_file_count} fichiers)")
                    
            except Exception as e:
                print(f"⚠️ Erreur cycle {cycles}: {e}")
            
            # Afficher résumé
            print(analyzer.get_summary())
            
            # Attendre avant prochain cycle
            remaining_time = (duration_minutes * 60) - (time.time() - start_time)
            wait_time = min(check_interval, remaining_time)
            
            if wait_time > 0:
                print(f"⏸️ Attente {wait_time:.0f}s...")
                time.sleep(wait_time)
        
        print(f"\n✅ Mode continu terminé après {cycles} cycles")
        
    except KeyboardInterrupt:
        print(f"\n⏹️ Mode continu interrompu après {cycles} cycles")
    except Exception as e:
        print(f"\n❌ Erreur mode continu: {e}")

# Interface pour lancer le mode continu
print("\n🎯 MODE CONTINU DISPONIBLE")
print("Pour activer la surveillance continue:")
print("   run_continuous_analysis_robust(duration_minutes=15)")
print("\n💡 Le mode continu surveille les nouvelles données et met à jour le feedback automatiquement")

## 🎯 Guide d'utilisation robuste

### ✅ Fonctionnalités de cette version

1. **Setup robuste** - Gestion automatique des credentials Git
2. **Analyse étendue** - 9 dhātus + scoring de qualité
3. **Visualisations** - Graphiques automatiques
4. **Feedback intelligent** - Recommandations adaptées
5. **Synchronisation sécurisée** - Pas d'erreurs fatales
6. **Mode continu** - Surveillance automatique

### 🚀 Ordre d'exécution recommandé

1. **Setup robuste** ← Commencez ici
2. **Analyseur dhātu** ← Initialisation
3. **Chargement données** ← Analyse du corpus
4. **Visualisation** ← Graphiques des résultats
5. **Feedback** ← Communication avec collecteur
6. **Mode continu** ← (Optionnel) Surveillance

### 🔧 Résolution des problèmes

- **Git non configuré** → Configuration automatique
- **Pas de données** → Exemples fournis
- **Erreurs push** → Sauvegarde locale garantie
- **Timeout** → Gestion gracieuse

🎯 **Cette version évite toutes les erreurs fatales !**